In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import chaospy as cp
import numpy as np
import pytest
from scipy.stats import norm
from statsmodels.stats.correlation_tools import corr_nearest

from econsa.copula import _cov2corr
from econsa.correlation import gc_correlation


def get_strategies(name):
    dim = np.random.randint(2, 10)
    means = np.random.uniform(-100, 100, dim)

    if name == "test_gc_correlation_functioning":
        # List of distributions to draw from.
        # Repeated distributions are for higher drawn frequency, not typo
        distributions = [
            cp.Exponential,
            cp.Gilbrat,
            cp.HyperbolicSecant,
            cp.Laplace,
            cp.LogNormal,
            cp.LogNormal,
            cp.LogNormal,
            cp.LogUniform,
            cp.LogUniform,
            cp.LogUniform,
            cp.LogWeibull,
            cp.Logistic,
            cp.Maxwell,
            cp.Normal,
            cp.Normal,
            cp.Normal,
            cp.Rayleigh,
            cp.Uniform,
            cp.Uniform,
            cp.Uniform,
            cp.Wigner,
        ]
        marginals = list()
        for mean in means:
            dist = distributions[np.random.choice(len(distributions))](mean)
            marginals.append(dist)

        cov = np.random.uniform(-1, 1, size=(dim, dim))
        cov = cov @ cov.T
        # If not positive definite, find the nearest one.
        if np.all(np.linalg.eigvals(cov) > 0) == 0:
            cov = corr_nearest(cov)

        corr = _cov2corr(cov).round(8)
    elif (
        name == "test_gc_correlation_2d" or name == "test_gc_correlation_2d_force_calc"
    ):
        dim = 2
        means = np.random.uniform(-100, 100, dim)
        distributions = [
            cp.Normal,
            cp.Uniform,
            cp.Exponential,
            cp.Rayleigh,
            cp.LogWeibull,
        ]
        marginals = [cp.Normal(means[0])]
        dist2 = distributions[np.random.choice(len(distributions))](means[1])
        marginals.append(dist2)

        cov = np.random.uniform(-1, 1, size=(dim, dim))
        cov = cov @ cov.T
        # If not positive definite, find the nearest one.
        if np.all(np.linalg.eigvals(cov) > 0) == 0:
            cov = corr_nearest(cov)

        corr = _cov2corr(cov).round(8)
    elif name == "test_gc_correlation_exception_marginals":
        marginals = list()
        for i in range(dim):
            marginals.append(norm())

        corr = np.random.uniform(-1, 1, size=(dim, dim))
    elif name == "test_gc_correlation_exception_corr_symmetric":
        distributions = [cp.Normal, cp.Uniform, cp.LogNormal]
        marginals = list()
        for mean in means:
            dist = distributions[np.random.choice(len(distributions))](mean)
            marginals.append(dist)

        corr = np.random.uniform(-1, 1, size=(dim, dim))
    else:
        raise NotImplementedError

    strategy = (marginals, corr)
    return strategy



In [26]:
for _ in range(100):
    print(_)
    np.random.seed(_)
    marginals, corr = get_strategies("test_gc_correlation_2d_force_calc")
    corr_ref_numbers = gc_correlation(marginals, corr)
    corr_force_calc = gc_correlation(marginals, corr, force_calc=True)

    
    print(corr_ref_numbers.round(4))
    print(corr_force_calc.round(4))

0
[[1.     0.7317]
 [0.7317 1.    ]]
[[1.    0.733]
 [0.733 1.   ]]
1


KeyboardInterrupt: 

In [4]:
np.random.seed(2)
marginals, corr = get_strategies("test_gc_correlation_2d_force_calc")
corr_ref_numbers = gc_correlation(marginals, corr)
corr_force_calc = gc_correlation(marginals, corr, force_calc=True)

KeyboardInterrupt: 

In [47]:
corr

array([[1.        , 0.84663634],
       [0.84663634, 1.        ]])

In [35]:
corr_force_calc

array([[ 1.00000000e+00, -2.81817373e-06],
       [-2.81817373e-06,  1.00000000e+00]])

In [36]:
corr_ref_numbers

array([[1.        , 0.93722643],
       [0.93722643, 1.        ]])

In [9]:
n_points = 100
distribution = cp.MvNormal(loc=np.zeros(2), scale=np.eye(2))
draws = distribution.sample(n_points, rule="S").T.reshape(n_points, 2)

In [16]:

x, y = draws[:, 0], draws[:, 1]